# Demo Konzept Virtualfeed
Dies ist eine kleine Demo die die Implementation des Virtual feeds erklären und demonstrieren soll

Wir haben drei devices mit eigenen RSA Schlüsseln. Der Virtuelle Feed wird in den drei Devicefeeds gespeichert und existiert somit nur virtuell. Der Virtuelle Feed hat jedoch wie die Hostfeeds (= device 1-3) einen eigenen RSA schlüssel.

In [1]:
e = [193,475,79,89,47] #public Key element
d = [62785,21307,303,1865,283] #private Key element
M = [124373,21971,3151,2291,781] # Key Modul M
device1 = ["device1","pubkey0"]
device2 = ["device2","pubkey1"]
device3 = ["device3","pubkey2"]
deviceList = [device1,device2,device3]
mesgPointer = [0]*100
virtualFeeds = ['']*100
vFeedPointer = 0

### Simple RSA Ver - und Entschlüsslung:
Disclaimer: das ist lediglich eine Demoimplementierung von RSA, sie basiert auf der Theoretischen Grundlage von RSA, ist aber nicht sicher, da die Schlüsselzahlen zu klein sind...


In [2]:
def textEnc(key,text):
  eT = ""
  for i in text:
    eT  = eT + chr((ord(i)**d[key])%M[key])
  #print("encrypted with: ",key)
  return eT

In [3]:
def textDec(key,enctext):
  dT = ""
  for i in enctext:
    dT  = dT + chr((ord(i)**e[key])%M[key])
  #print("decrypted with: ",key)
  return dT

In [4]:
textEnc(0,"Hallo")

'达⚔\U0001e2bf\U0001e2bf\U00012a56'

In [5]:
textDec(0,"达⚔\U0001e2bf\U0001e2bf\U00012a56")

'Hallo'

In [6]:
#FunctionCheck for the RSA numbers
leng = len(e)
print("correct transformation over 1:")
for i in range(0,leng,1):
  enc = textEnc(i,"abcdefghijklmnopqrstuvwxyzöäü,.-:")
  dec = textDec(i,enc)
  #print(dec)
  print(dec == "abcdefghijklmnopqrstuvwxyzöäü,.-:")

print("simlpe oclusion test:")
for i in range(0,leng,1):
  for j in range(0,leng,1):
    enc = textEnc(i,"abcdefghijklmnopqrstuvwxyzöäü")
    dec = textDec(j,enc)
    if(dec == "abcdefghijklmnopqrstuvwxyzöäü"):
      print(i,j)

print("double oclusion test:")
for i in range(0,leng,1):
  for j in range(0,leng,1):
    enc = textEnc(i,"abcdefghijklmnopqrstuvwxyzöäü")
    enc = textEnc(j,enc)
    dec = textDec(i,enc)
    if(dec == "abcdefghijklmnopqrstuvwxyzöäü"):
      print(i,j)

print("incorrect backtransformation over 2:")
for i in range(0,leng,1):
  for j in range(0,leng,1):
    enc = textEnc(i,"abcdefghijklmnopqrstuvwxyzöäü")
    enc = textEnc(j,enc)
    dec = textDec(j,enc)
    dec = textDec(i,dec)
    if(dec != "abcdefghijklmnopqrstuvwxyzöäü"):
      print(i,j)

correct transformation over 1:
True
True
True
True
True
simlpe oclusion test:
0 0
1 1
2 2
3 3
4 4
double oclusion test:
incorrect backtransformation over 2:
0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


### Virtual Feed Funktionen


In [7]:
import random
def newVirtualChat(name,devices):
  global virtualFeeds
  global vFeedPointer
  key = vFeedPointer + 3 #jump over the feed keys
  virtualFeeds[vFeedPointer] = name
  deviceNames = []
  for i in devices:
    header = "V"+str(vFeedPointer)+ ":"+name
    pubkey = "V"+str(vFeedPointer)+":pubkey"+str(key)
    newfeedMsg(i,header)
    newfeedMsg(i,pubkey)
    deviceNames.append(i[0])
  vFeedPointer +=1
  print("NewVirtualChat created: ",name, "devices: ",deviceNames, "pubkey e:",e[key],"M:",M[key])

In [8]:
def printFeed(feed):
  key = int(feed[1][6:7])
  for i in range(0,2,1):
    print(feed[i])
  for i in range(2,len(feed),1):
    print(textDec(key,feed[i]))

In [9]:
def printRawFeed(feed):
  for i in feed:
    print(i)

In [10]:
def printVirtualFeedHeader(feed):
  for i in deviceList:
    for j in i:
      if (j == "V"+feed):
        print(j)

In [11]:
def printVirtualFeed(feed):
  global vFeedPointer
  mesgList = ['']*100;
  index = -1
  messOn = False
  vKey = -1
  s = 0
  for i in range(0,vFeedPointer,1):
    if(virtualFeeds[i]== feed):
      vKey = i + 3;
      #print(vKey)
  for i in deviceList:
    dKey = int(i[1][6:7])
    for j in i:
      if (textDec(dKey,j[1:3+len(feed)]) == "V:"+feed):
         index = ord(textDec(dKey,j[0]))-48
         mesgList[index] = textDec(dKey,j[4+len(feed):])
  for m in mesgList:
    if (m !=''):
      print(textDec(vKey,m))

In [12]:
def newfeedMsg(feed,message):
  feed.append(textEnc(int(feed[1][6:7]),message))

In [13]:
newfeedMsg(device1,"hallo")

In [14]:
def newVirtualMesg(vfeed,dfeed,message): #gehört zu Virtualize
  global mesgPointer
  global virtualFeeds
  global vFeedPointer
  mp = -1;
  vKey = -1;
  for i in range(0,vFeedPointer,1):
    if(virtualFeeds[i]== vfeed):
      mp = mesgPointer[i]
      mesgPointer[i] +=1
      vKey = i + 3
      #print(vKey)
  if(mp<0):
    print("Du musst zuerst einen Virtuellen Feed erstellen")
  else:
    message = textEnc(vKey,message)
    msg = str(mp)+"V:"+vfeed+":"+message
    newfeedMsg(dfeed,msg)


### erstellen eines Virtuellen Chats "virtualtest" auf drei geräten:



In [15]:
#erstellen eines Virtuellen Chats "virtualtest" auf drei geräten 
name = "virtualtest"
devices = [device1,device2,device3]
newVirtualChat(name, devices)

NewVirtualChat created:  virtualtest devices:  ['device1', 'device2', 'device3'] pubkey e: 89 M: 2291


### Erstellen von neuen Nachrichten auf verschiedenen Geräten

In [16]:
#erstellen einer neuen nachricht in "virtualtest" von gerät1
vfeed = "virtualtest"
dfeed = device1
message = "hello world"
newVirtualMesg(vfeed,dfeed,message)

In [17]:
#erstellen einer neuen nachricht in "virtualtest" von gerät1
vfeed = "virtualtest"
dfeed = device1
message = "damit nicht das \"Fork\" problem auftritt, muss ich das Schreibrecht an Gerät 2 weitergeben "
newVirtualMesg(vfeed,dfeed,message)

In [18]:
#erstellen einer neuen nachricht in "virtualtest" von gerät2
vfeed = "virtualtest"
dfeed = device2
message = "Dann kann ich, Gerät2 auf den Virtualfeed schreiben"
newVirtualMesg(vfeed,dfeed,message)

### Was in den einzelnen Geräte - Feeds steht:

In [19]:
#Feed von Gerät 1
printRawFeed(device1)

device1
pubkey0
𒆪⚔𞊿𞊿𒩖
𗵢鏾𓶯𗭊똀𝫖𖣆⚔𞊿忎𐱒
𗵢鏾𓶯𖣆𓪭⮿忎𘚩䛪
鏾𗵢𓶯𗭊똀𝫖𖣆⚔𞊿忎𐱒𓶯𒬼𑤀══囔붭𖅲囔楼═괋
𑾔𗵢𓶯𗭊똀𝫖𖣆⚔𞊿忎𐱒𓶯괋ڡ𗨙镹𐺡붭𗚷镹𒫢𒬼𐺡붭괋ڡ𓏈붭鬒ꄑ囔楼鞺鬒붭꣝楼囔硖═𑤀𗨙붭ڡℌ𑕝𐺡楼镹𐺡𐺡㬴붭𗨙ℌ𓏈𓏈붭镹𒫢𒬼붭괋ڡ𓏈붭𒖖𒫢𒬼楼𑤀镹硖楼𑤀𒫢𒬼𐺡붭ڡ𗚷붭훪𑤀楼𘕢𐺡붭붭𖅲𑤀镹𐺡𑤀楼𙜜𑤀硖𑤀𗚷붭


In [20]:
#Feed von Gerät 1
printFeed(device1)

device1
pubkey0
hallo
V0:virtualtest
V0:pubkey3
0V:virtualtest:چԤࡘࡘѢ˗ޙѢ֕ࡘش
1V:virtualtest:شڡȏ֧è˗ֺ֧ǹچè˗شڡࠊ˗ϣ̸Ѣ֕äϣ˗ࢳ֕ѢӅࡘԤȏ˗ڡźè֧֕èèҚ˗ȏźࠊࠊ˗֧ǹچ˗شڡࠊ˗ׁǹچ֕Ԥ֧Ӆ֕Ԥǹچè˗ڡֺ˗ʱԤ֕ƭè˗ך˗ޙԤ֧èԤ֕շԤӅԤֺ˗


In [21]:
#Feed von Gerät 2
printFeed(device2)

device2
pubkey1
V0:virtualtest
V0:pubkey3
2V:virtualtest:٧ڡֺֺ˗äڡֺֺ˗֧ǹچҚ˗ʱԤ֕ƭèך˗ڡź˗شԤֺ˗ϙ֧֕èźڡࡘԤԤش˗ࠊǹچ֕Ԥ֧ӅԤֺ


In [22]:
#Feed von Gerät 3
printFeed(device3)

device3
pubkey2
V0:virtualtest
V0:pubkey3


### Was im Virtual Feed steht:

In [23]:
printVirtualFeed(vfeed)

hello world
damit nicht das "Fork" problem auftritt, muss ich das Schreibrecht an Gerät 2 weitergeben 
Dann kann ich, Gerät2 auf den Virtualfeed schreiben
